# Function Calling with GitHub Marketplace Models

This notebook demonstrates how to implement and use function calling with Language Learning Models (LLMs) via GitHub Marketplace Models. Function calling allows AI models to request specific external tools or data sources when they need additional information beyond their training data.

## Learning Objectives:

1. Understand what function calling is and why it's important for extending LLM capabilities
2. Compare standard chat completions with function-enhanced responses
3. Implement the complete function calling workflow:
   - Define function schemas that the model can use
   - Send prompts with available tools
   - Parse function call requests from the model
   - Execute the requested functions with proper arguments
   - Return results to the model for final response generation

This notebook uses a weather function example to show how models can access real-time data that isn't in their training.

# Function Calling 

The LLM models are limited to generation based on knowledge included in their training data or context information provided in the prompt (see RAG concepts). 

However, function calling allows the model extend it's capabilities and request to invoke function to perform specific tasks or retrieve information. 

This is particularly useful for tasks that require structured data or specific actions, such as retrieving information from a database or performing calculations.


References: 
- https://platform.openai.com/docs/guides/function-calling
- https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models

In [9]:
import os
import json
from dotenv import load_dotenv
from openai import AsyncOpenAI, OpenAI
from agents import (
    Agent,
    Runner,
    OpenAIChatCompletionsModel,
    ModelProvider,
    Model,
    RunConfig,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled,
    function_tool,
)
load_dotenv()

True

In [10]:
client = AsyncOpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

client2 = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.environ["GITHUB_TOKEN"],
)

In [11]:
def message(role, content):
    return {"role": role, "content":
            [{
                "type": "text",
                "text": content}]
    }


In [12]:
messages = [
    message("developer", "you are a helpful assistant"),
    message("user", "How is the weather in Sydney today?"),
]

## Standard Chat Completions call 

LLM has no context of realtime data or information not currently in its training data.

In [13]:

completion = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages)



print(completion.choices[0].message.to_json())

{
  "content": "I'm unable to provide real-time weather updates. I recommend checking a reliable weather website or using a weather app for the most current information on Sydney's weather.",
  "refusal": null,
  "role": "assistant"
}


## Function Calling Example 

The flow of function calling is as follows:

<img src="https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png" alt="Function Calling" width="400px"/>

### Define the function

In [14]:
get_weather_function_definition = {
    "type": "function",
    "function":{
        "name": "get_weather",
        "description": "Get the current weather in a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The name of the location to get the weather for",
                }
            },
            "required": ["location"],
            "additionalProperties": False,

        },
    },
    "strict": True
}

tools = [get_weather_function_definition]

In [15]:
def get_weather(location):
    # return f"The weather in {location} is sunny with a high of 25°C."
    return {"temperature": 25, "condition": "sunny"}

### 1. Send messasge with tools

In [17]:
chat_history = messages
completion = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=chat_history,
    tools=tools
)

### 2. Model responsds with function call 

In [18]:
# print(completion.to_json())

print(completion.choices[0].message.to_json())

{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "tool_calls": [
    {
      "id": "call_A0GbXBeZGkAR1ooDoMKQCPcs",
      "function": {
        "arguments": "{\"location\":\"Sydney\"}",
        "name": "get_weather"
      },
      "type": "function"
    }
  ]
}


must append the function call request to the message history, then perform the requested function call. 

In [19]:
chat_history.append(json.loads(completion.choices[0].message.to_json()))  # append model's function call message
chat_history

[{'role': 'developer',
  'content': [{'type': 'text', 'text': 'you are a helpful assistant'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'How is the weather in Sydney today?'}]},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_A0GbXBeZGkAR1ooDoMKQCPcs',
    'function': {'arguments': '{"location":"Sydney"}', 'name': 'get_weather'},
    'type': 'function'}]}]

### 3. Execute Function call 

Inspect the finish_reson, if it is "tool_calls", then proceed to execute the request function with arguments specified by the LLM function call request

In [20]:
finish_reason = completion.choices[0].finish_reason
print(finish_reason)
tool_calls = completion.choices[0].message.tool_calls
print(tool_calls[0].to_json())



tool_calls
{
  "id": "call_A0GbXBeZGkAR1ooDoMKQCPcs",
  "function": {
    "arguments": "{\"location\":\"Sydney\"}",
    "name": "get_weather"
  },
  "type": "function"
}


Execute the fucntion, and append function call result to the message history. Must tool_call_id returned by the LLM function call request.

In [21]:

for tool in tool_calls:
    print(tool.id)
    print(tool.function.name)
    print(tool.function.arguments)
    function_name = tool.function.name
    function_args = json.loads(tool.function.arguments)
    tool_call_id = tool.id



    # execute the function call and append the results as a ToolMessage in the message history
    function_call_results = locals()[function_name](**function_args)
    print(function_call_results)
    function_call_results_message = {
        "role": "tool",
        "type": "function_call_output",
        "tool_call_id": tool_call_id,
        "content": str(function_call_results)
    }


    chat_history.append(function_call_results_message)

call_A0GbXBeZGkAR1ooDoMKQCPcs
get_weather
{"location":"Sydney"}
{'temperature': 25, 'condition': 'sunny'}


In [22]:
str(function_call_results)

"{'temperature': 25, 'condition': 'sunny'}"

In [23]:
chat_history

[{'role': 'developer',
  'content': [{'type': 'text', 'text': 'you are a helpful assistant'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'How is the weather in Sydney today?'}]},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_A0GbXBeZGkAR1ooDoMKQCPcs',
    'function': {'arguments': '{"location":"Sydney"}', 'name': 'get_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'type': 'function_call_output',
  'tool_call_id': 'call_A0GbXBeZGkAR1ooDoMKQCPcs',
  'content': "{'temperature': 25, 'condition': 'sunny'}"}]

### 4. Send the result back to the model

Send the result of the function call back to the model as a message. This message should include the function call result and message history. Function call result must correctly include tool_call_id, so that the model can match the result with the original function call request.




In [24]:
response_2 = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=chat_history,
    tools=tools
)



### 5. Model final response 


The model will then respond with a message that includes the result of the function call. This message can be used to continue the conversation or provide additional information to the user.

In [28]:
print(response_2.choices[0].message.content)

The weather in Sydney today is sunny with a temperature of 25°C.


In [29]:
print(response_2.choices[0].message.to_json())

{
  "content": "The weather in Sydney today is sunny with a temperature of 25°C.",
  "refusal": null,
  "role": "assistant"
}


### 6. Continue the conversation

In [36]:
# always append model response to the chat history
chat_history.append(json.loads(response_2.choices[0].message.to_json()))  # append model's function call message
# new user message
chat_history.append(message("user", "what again in Arabic please?"))
response_3 = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=chat_history,
    tools=tools
)


print(response_3.choices[0].message.to_json())

{
  "content": "الطقس في سيدني اليوم مشمس ودرجة الحرارة 25°C.",
  "refusal": null,
  "role": "assistant"
}
